In [572]:
import os
import sklearn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression as sk_OLS
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
import math

from sklearn.metrics import r2_score
import string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression

In [573]:
def scaller(edu):
    x_max = max(edu)
    x_min = min(edu)
    scaled_xs =[]
    for x in edu:
        x_scaled = 4*(x-x_min)/(x_max-x_min)+1
        scaled_xs.append(x_scaled)
    return scaled_xs

In [574]:
longitudinal = pd.read_csv("datacsv/oasis_longitudinal.csv")
cross_sectional = pd.read_csv("datacsv/oasis_cross-sectional.csv")

edu = longitudinal.EDUC
scalled_edu = scaller(edu)
longitudinal["Educ"] = scalled_edu

frames = [cross_sectional, longitudinal]


combined = pd.concat(frames)
# combined = combined.rename(columns={"Group": "CDR"})
# combined = combined.map(rename)
# combined["Group"] = combined["Group"].astype('float')
# print(combined)
combined.reset_index(inplace=True, drop=True)


print(combined)
combined.to_csv("combined.csv")

                ID M/F Hand  Age      Educ  SES  MMSE  CDR  eTIV   nWBV  \
0    OAS1_0001_MR1   F    R   74  2.000000  3.0  29.0  0.0  1344  0.743   
1    OAS1_0002_MR1   F    R   55  4.000000  1.0  29.0  0.0  1147  0.810   
2    OAS1_0003_MR1   F    R   73  4.000000  3.0  27.0  0.5  1454  0.708   
3    OAS1_0004_MR1   M    R   28       NaN  NaN   NaN  NaN  1588  0.803   
4    OAS1_0005_MR1   M    R   18       NaN  NaN   NaN  NaN  1737  0.848   
..             ...  ..  ...  ...       ...  ...   ...  ...   ...    ...   
804            NaN   M    R   82  3.352941  1.0  28.0  0.5  1693  0.694   
805            NaN   M    R   86  3.352941  1.0  26.0  0.5  1688  0.675   
806            NaN   F    R   61  2.647059  2.0  30.0  0.0  1319  0.801   
807            NaN   F    R   63  2.647059  2.0  30.0  0.0  1327  0.796   
808            NaN   F    R   65  2.647059  2.0  30.0  0.0  1333  0.801   

       ASF  Delay Subject ID         MRI ID        Group  Visit  MR Delay  \
0    1.306    NaN     

In [575]:
# combined['M/F_new'], categories = pd.factorize(combined["M/F"])
# combined['Hand_new'], categories = pd.factorize(combined["Hand"])

In [576]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

cdr_na = combined["CDR"].isna()
# print(combined[cdr_na])


unlabled = combined[cdr_na]
labled = combined[~cdr_na]


print(len(labled)/len(combined))

y_label = labled.CDR
X_label = labled.drop([ "M/F", "Hand", "Subject ID", "MRI ID", "ID", "Group", "Visit", "EDUC", "Delay", "ASF", "CDR", "MR Delay"], axis=1)
# X_cluster_label = X_label.drop(["MR Delay", "SES", "MMSE"], axis=1)

# , "SES", "MMSE", "ASF"
x_unlabeled = unlabled.drop([ "M/F", "Hand", "Subject ID", "MRI ID", "ID", "Group", "Visit", "EDUC", "Delay", "ASF", "CDR", "MR Delay"], axis=1)
print(X_label)


0.7515451174289246
     Age      Educ  SES  MMSE  eTIV   nWBV
0     74  2.000000  3.0  29.0  1344  0.743
1     55  4.000000  1.0  29.0  1147  0.810
2     73  4.000000  3.0  27.0  1454  0.708
8     74  5.000000  2.0  30.0  1636  0.689
9     52  3.000000  2.0  30.0  1321  0.827
..   ...       ...  ...   ...   ...    ...
804   82  3.352941  1.0  28.0  1693  0.694
805   86  3.352941  1.0  26.0  1688  0.675
806   61  2.647059  2.0  30.0  1319  0.801
807   63  2.647059  2.0  30.0  1327  0.796
808   65  2.647059  2.0  30.0  1333  0.801

[608 rows x 6 columns]


In [577]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(scaler.fit(X_label))
print(scaler.mean_)
X_label = scaler.transform(X_label)
# np.nan_to_num(X_scaled, copy=False, nan=-1)
# X_scaled.nan_to_num(value=-1, inplace=True)
print(X_label)

def scale_non_cat(x):
    scaler = StandardScaler()
    x_s = x["eTIV",   "nWBV"]
    print(scaler.fit(x_s))

    x_s = scaler.transform(X_label)
    # np.nan_to_num(X_scaled, copy=False, nan=-1)
    # X_scaled.nan_to_num(value=-1, inplace=True)
    print(x_s)
    x["eTIV",   "nWBV"]

    


StandardScaler()
[7.52088816e+01 3.08320433e+00 2.47192982e+00 2.72343234e+01
 1.47706250e+03 7.37129934e-01]
[[-0.12264306 -1.11252064  0.46853768  0.47916067 -0.7803639   0.13768155]
 [-2.05022486  0.94160821 -1.30599042  0.47916067 -1.93569835  1.70915693]
 [-0.22409473  0.94160821  0.46853768 -0.06358955 -0.1352533  -0.68323842]
 ...
 [-1.44151482 -0.44794954 -0.41872637  0.75053578 -0.92697995  1.49806322]
 [-1.23861147 -0.44794954 -0.41872637  0.75053578 -0.88006282  1.38078894]
 [-1.03570812 -0.44794954 -0.41872637  0.75053578 -0.84487496  1.49806322]]


In [578]:
# normalize values:
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer


imp_mean = IterativeImputer(random_state=42)
X_label = imp_mean.fit_transform(X_label)
X_label = pd.DataFrame(X_label, columns=["Age","Educ",  "SES",  "MMSE",  "eTIV", "nWBV"])

X = x_unlabeled
x_infered_labels = imp_mean.transform(X)
x_infered_labels = pd.DataFrame(x_infered_labels, columns=["Age", "Educ", "SES", "MMSE", "eTIV", "nWBV"])
frames = [X_label, x_infered_labels]


combined = pd.concat(frames)
# y_label = combined.CDR
# combined = combined.drop(["CDR"], axis=1)


/usr/local/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but IterativeImputer was fitted without feature names
  warnings.warn(


In [579]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
x_components = pca.fit_transform(X_label)
# print(pca.explained_variance_ratio_)
# print(pca.singular_values_)

In [580]:
X_train, X_test, y_train, y_test = train_test_split(x_components, y_label)
print(len(X_train))
# print(y_train)
print(len(X_test))
print(y_label)
# print(X_test)
y_train = y_train.astype('int')
y_test = y_test.astype('int')

456
152
0      0.0
1      0.0
2      0.5
8      0.0
9      0.0
      ... 
804    0.5
805    0.5
806    0.0
807    0.0
808    0.0
Name: CDR, Length: 608, dtype: float64


In [581]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf_svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf_svm.fit(X_train, y_train)
y_pred = clf_svm.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))




# y_unlabeled = clf_svm.predict(x_unlabeled)
# print(y_unlabeled)

Accuracy:  0.9144736842105263


In [582]:
x_un_components = pca.fit_transform(x_infered_labels)

y_unlabeled = clf_svm.predict(x_un_components)
print(y_unlabeled)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [583]:
from xgboost import XGBClassifier

# create model instance
# print(y_label)
# X_train, X_test, y_train, y_test = train_test_split(combined, y_label)
bst = XGBClassifier(n_estimators=112, max_depth=7, learning_rate=.5, objective='reg:quantileerror')
# fit model
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))

y_unlabeled = bst.predict(x_un_components)
print(y_unlabeled)
print()

Accuracy:  0.9342105263157895
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

